In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

data = pd.read_csv("job_applicant_data.csv")

In [ ]:
data

,Unnamed: 0,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,51552.0,C++;Python;Git;PostgreSQL,4,0
1,1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,46482.0,Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,12,1
2,2,<35,No,Master,1,Man,No,Dev,15,6,Germany,77290.0,C;C++;Java;Perl;Ruby;Git;Ruby on Rails,7,0
3,3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,46135.0,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL;Gi...,13,0
4,4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,160932.0,C++;Python,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,73457,<35,No,Undergraduate,1,Man,No,Dev,7,2,Germany,41058.0,C#;HTML/CSS;JavaScript;TypeScript;Docker;Kuber...,13,1
73458,73458,>35,No,Undergraduate,1,Man,No,Dev,21,16,United States of America,115000.0,C#;HTML/CSS;Java;JavaScript;npm;ASP.NET Core ;...,11,1
73459,73459,<35,No,Undergraduate,1,Man,No,Dev,4,3,Nigeria,57720.0,HTML/CSS;JavaScript;TypeScript;Docker;Express;...,12,1
73460,73460,<35,Yes,Undergraduate,1,Man,Yes,Dev,5,1,United States of America,70000.0,C#;HTML/CSS;JavaScript;SQL;TypeScript;npm;Yarn...,15,1


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Drop irrelevant columns
data = data.drop(columns=['Unnamed: 0'])

# Create a new feature based on number of skills
data['NumTechWorkedWith'] = data['HaveWorkedWith'].str.count(';') + 1

# Drop HaveWorkedWith
data = data.drop(columns=['HaveWorkedWith'])

# Encode Gender:
gender_mapping = {'Man': 1, 'Woman': 0, 'NonBinary': 2}
inv_gender_mapping = {v: k for k, v in gender_mapping.items()}
data['Gender'] = data['Gender'].map(gender_mapping)

# Encode Age:
age_mapping = {'>35': 1, '<35' : 0}
inv_age_mapping =  {v: k for k, v in age_mapping.items()}
data['Age'] = data['Age'].map(age_mapping)

# Encode EdLevel:
ed_level_mapping = {'PhD': 4, 'Master' : 3, 'Undergraduate': 2, 'NoHigherEd': 1, 'Other': 0}
inv_ed_level_mapping  =  {v: k for k, v in ed_level_mapping.items()}
data['EdLevel'] = data['EdLevel'].map(ed_level_mapping)

# Encode categorical variables
label_encoders = {}
for column in ['Age', 'Accessibility', 'MentalHealth', 'MainBranch', 'Country']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column].astype(str))
    label_encoders[column] = le


# Split the data into features and target
X = data.drop(columns=['Employed'])
y = data['Employed']

In [ ]:
# print(X)
S = data["Gender"]
# X = X.drop(columns=['Gender'])
print(S.value_counts())
X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X, y, S, test_size=0.2, random_state=42)
X_train_W_Gender = X_train
X_test_W_Gender = X_test
X_train = X_train.drop(columns=['Gender'])
X_test = X_test.drop(columns=['Gender'])

Gender
1    68573
0     3518
2     1371
Name: count, dtype: int64


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(S_train.shape)

(58769, 12)
(58769,)
(58769,)


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42, min_samples_split=5, min_samples_leaf=5, max_features='log2', max_depth=20)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=5,
                       min_samples_split=5, random_state=42)

In [ ]:
y_pred_train = rf.predict_proba(X_train)[:, 1]
y_pred_test = rf.predict_proba(X_test)[:, 1]

y_pred_train_binary = (y_pred_train > 0.5).astype(int)
y_pred_test_binary = (y_pred_test > 0.5).astype(int)

print("Train Accuracy:", accuracy_score(y_train, y_pred_train_binary))
print("Test Accuracy:", accuracy_score(y_test, y_pred_test_binary))

Train Accuracy: 0.8437441508278174
Test Accuracy: 0.7844551827400803


In [ ]:
class Adversary(nn.Module):
    def __init__(self):
        super(Adversary, self).__init__()
        self.fc1 = nn.Linear(1, 16)  # ✅ Input should be (batch_size, 1)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)  # ✅ Output = 3 classes (0,1,2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # 🚀 No softmax, CrossEntropyLoss handles it
        return x

# Initialize Adversarial Model
adv_model = Adversary()
adv_optimizer = optim.Adam(adv_model.parameters(), lr=0.01)
adv_criterion = nn.CrossEntropyLoss()  # 🚀 Multi-class loss

# Convert classifier predictions to PyTorch tensor (Ensure correct shape)
y_pred_tensor = torch.tensor(y_pred_train.reshape(-1, 1), dtype=torch.float32)

# Convert sensitive attribute (Gender) to `long` type for multi-class classification
sensitive_tensor = torch.tensor(S_train.values, dtype=torch.long)

# Train the Adversary
for epoch in range(500):
    adv_optimizer.zero_grad()

    s_pred = adv_model(y_pred_tensor)  # Predict sensitive attribute

    # Compute adversarial loss
    loss = adv_criterion(s_pred, sensitive_tensor)  # ✅ Works for 3 classes!
    loss.backward()
    adv_optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Adversarial Loss: {loss.item()}")


y_pred_tensor shape BEFORE FIX: torch.Size([58769, 1])
y_pred_tensor shape AFTER FIX: torch.Size([58769, 1])
Epoch 0, Adversarial Loss: 1.3846197128295898
Epoch 100, Adversarial Loss: 0.283485472202301
Epoch 200, Adversarial Loss: 0.28302451968193054
Epoch 300, Adversarial Loss: 0.282814085483551
Epoch 400, Adversarial Loss: 0.28275057673454285


In [ ]:
# Put adversarial model in evaluation mode
adv_model.eval()

# Compute adversary's predicted probabilities for gender
s_pred_train = adv_model(y_pred_tensor).detach().numpy()

# Convert adversary outputs into probabilities (softmax)
s_pred_train = np.exp(s_pred_train) / np.exp(s_pred_train).sum(axis=1, keepdims=True)

# Compute bias correction (difference between true and predicted sensitive attributes)
bias_correction = S_train.values - np.argmax(s_pred_train, axis=1)

# Print some values for debugging
print("Bias Correction Sample:", bias_correction[:10])


Bias Correction Sample: [ 0  0  0  0  0  0  0  0 -1  0]


In [ ]:
# Define fairness trade-off (higher = more fairness, lower = more accuracy)
lambda_fairness = 0.9 # Adjust this value to control fairness impact

group_means = S_train.groupby(S_train).transform('mean')
# print(group_means)

# Compute bias correction (difference between true and predicted sensitive attributes)
bias_correction = lambda_fairness * ((S_train.values - np.argmax(s_pred_train, axis=1))*group_means)

# Apply correction to labels
adjusted_labels = np.clip(y_train-(0.5*bias_correction), 0, 1)

# Print some values for debugging
print("Adjusted Labels Sample:", adjusted_labels[:10])


Adjusted Labels Sample: 32178    1.0
18457    1.0
47849    0.0
59358    1.0
37397    1.0
54482    1.0
65308    1.0
7579     0.0
40915    0.0
38777    1.0
dtype: float64


In [ ]:
# Convert adjusted labels back to integer class (0 or 1)
adjusted_labels_binary = (adjusted_labels > 0.5).astype(int)

# Retrain Random Forest using adjusted labels
rf_fair = RandomForestClassifier(
    n_estimators=100, random_state=42, min_samples_split=5,
    min_samples_leaf=5, max_features='log2', max_depth=20
)
rf_fair.fit(X_train, adjusted_labels_binary)

# Evaluate the fair model
y_pred_fair_test = rf_fair.predict_proba(X_test)[:, 1]
y_pred_fair_test_binary = (y_pred_fair_test > 0.5).astype(int)

accuracy_fair = accuracy_score(y_test, y_pred_fair_test_binary)

print(f"Fair Model Accuracy: {accuracy_fair:.4f}")


Fair Model Accuracy: 0.7848


In [ ]:
# Compute accuracy for biased and fair models
accuracy_unfair = accuracy_score(y_test, y_pred_test_binary)
accuracy_fair = accuracy_score(y_test, y_pred_fair_test_binary)

print(f"Original Model Accuracy: {accuracy_unfair:.4f}")
print(f"Fair Model Accuracy: {accuracy_fair:.4f}")


Original Model Accuracy: 0.7845
Fair Model Accuracy: 0.7848


In [ ]:
p_rule = (
    np.mean(y_pred_fair_test_binary[S_test == 1]) /
    np.mean(y_pred_fair_test_binary[S_test == 0])
) * 100
print(f"Demographic Parity (p-rule): {p_rule:.2f}%")


Demographic Parity (p-rule): 123.05%


In [ ]:
from collections import defaultdict

# Unique Gender Groups
unique_Gender_groups = data['Gender'].unique()
unique_Gender_groups.sort()

# # Initialize counters
# Gender_counts = defaultdict(int)
# positive_label_counts = defaultdict(int)

# # Count instances per gender group
# for Gender, pred in zip(X_test_W_Gender['Gender'], y_pred_fair_test_binary):
#     Gender_counts[Gender] += 1
#     if pred == 1:  # Count only positive predictions
#         positive_label_counts[Gender] += 1

# # Compute Demographic Parity
# demographic_parity = {}
# for i in unique_Gender_groups:
#     demographic_parity[i] = positive_label_counts[i] / Gender_counts[i]
#     print(f"Demographic Parity for Gender Group {inv_gender_mapping[i]}: {demographic_parity[i]:.4f}")

# print()
demographic_parity = {}
for i in unique_Gender_groups:
    demographic_parity[i] = np.mean(y_pred_fair_test_binary[X_test_W_Gender['Gender'] == i])
    print(f"Demographic Parity for Gender Group {inv_gender_mapping[i]}: {demographic_parity[i]:.4f}")



Demographic Parity for Gender Group Woman: 0.4574
Demographic Parity for Gender Group Man: 0.5628
Demographic Parity for Gender Group NonBinary: 0.5478


In [ ]:
# Compute p% rule violations
count_rule_violation = 0
demographic_parity_values = list(demographic_parity.values())

for i in range(len(demographic_parity_values)):
    for j in range(i + 1, len(demographic_parity_values)):
        ratio = demographic_parity_values[i] / demographic_parity_values[j]
        if ratio < 0.8 or ratio > 1.25:  # Adjusted for bidirectional fairness check
            count_rule_violation += 1

print(f"Number of p% rule violations: {count_rule_violation}")


Number of p% rule violations: 0


In [ ]:
positive_outcomes = defaultdict(int)
negative_outcomes = defaultdict(int)
positive_predn = defaultdict(int)
negative_predn = defaultdict(int)

for idx, label in enumerate(y_test):
    prediction = y_pred_fair_test_binary[idx]
    Gender_group = S_test.iloc[idx]  # Ensure correct gender mapping

    if prediction == label and label == 0:
        negative_outcomes[Gender_group] += 1
    elif prediction == label and label == 1:
        positive_outcomes[Gender_group] += 1
    if prediction == 0:
        negative_predn[Gender_group] += 1
    elif prediction == 1:
        positive_predn[Gender_group] += 1

print("\nPositive Predictive Value (PPV):")
for group in unique_Gender_groups:
    if positive_predn[group] > 0:
        ppv = positive_outcomes[group] / positive_predn[group]
        print(f"PPV for Gender Group {inv_gender_mapping[group]}: {ppv:.4f}")

print("\nNegative Predictive Value (NPV):")
for group in unique_Gender_groups:
    if negative_predn[group] > 0:
        npv = negative_outcomes[group] / negative_predn[group]
        print(f"NPV for Gender Group {inv_gender_mapping[group]}: {npv:.4f}")



Positive Predictive Value (PPV):
PPV for Gender Group Woman: 0.7826
PPV for Gender Group Man: 0.7883
PPV for Gender Group NonBinary: 0.7383

Negative Predictive Value (NPV):
NPV for Gender Group Woman: 0.7932
NPV for Gender Group Man: 0.7794
NPV for Gender Group NonBinary: 0.8618


In [ ]:
positive_outcomes = defaultdict(int)
positive_label_for_Gender_group = defaultdict(int)

for idx, label in enumerate(y_test):
    prediction = y_pred_fair_test_binary[idx]
    Gender_group = S_test.iloc[idx]

    if prediction == 1 and label == 1:
        positive_outcomes[Gender_group] += 1
    if label == 1:
        positive_label_for_Gender_group[Gender_group] += 1

equal_opportunity = {}

for group in unique_Gender_groups:
    if positive_label_for_Gender_group[group] > 0:
        equal_opportunity[group] = positive_outcomes[group] / positive_label_for_Gender_group[group]
        print(f"Equal Opportunity for Gender Group {inv_gender_mapping[group]}: {equal_opportunity[group]:.4f}")


Equal Opportunity for Gender Group Woman: 0.7613
Equal Opportunity for Gender Group Man: 0.8214
Equal Opportunity for Gender Group NonBinary: 0.8661


## 8. Context: Summarize the main contributions of the paper and its relevance to
your task.

In my work, I build on the adversarial fairness framework introduced in the paper, which aims to reduce bias in decision trees and ensemble models like Random Forests. The paper proposes training an adversarial neural network to predict sensitive attributes (e.g., gender) from the model’s predictions, exposing hidden biases in decision-making. Using this adversary, the authors introduce a bias correction mechanism, where training labels are adjusted to ensure fairer predictions across demographic groups while maintaining accuracy. They evaluate fairness improvements using Demographic Parity, Equalized Odds, and Predictive Parity, showing that adversarial debiasing effectively mitigates disparities in real-world datasets. I apply a similar approach in my task, leveraging adversarial bias correction to modify training labels in a Random Forest model, ensuring that gender does not unduly influence predictions while preserving overall model performance.